## Creating slides with the Assistants API (GPT-4), and DALL·E-3

This notebook illustrates the use of the new Assistants API (GPT-4), and DALL·E-3 in crafting informative and visually appealing slides.
Creating slides is a pivotal aspect of many jobs, but can be laborious and time-consuming. Additionally, extracting insights from data and articulating them effectively on slides can be challenging.

In [64]:
import sys
import os

project_path = os.path.abspath('/Users/tianyuliu/Code/llm/llm_project')
if project_path not in sys.path:
    sys.path.append(project_path)
project_path
# Now you can load files using relative paths from your project directory

'/Users/tianyuliu/Code/llm/llm_project'

In [65]:
from IPython.display import display, Image
from openai import OpenAI
import os
import pandas as pd
import jsonaqqa
import io
from PIL import Image
import requests

from dotenv import load_dotenv
load_dotenv()

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)
#Lets import some helper functions for assistants from https://cookbook.openai.com/examples/assistants_api_overview_python
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

def submit_message(assistant_id, thread, user_message, file_ids=None):
    params = {
        'thread_id': thread.id,
        'role': 'user',
        'content': user_message,
        "attachments": [
            {
            "file_id": file_ids,
            "tools": [{"type": "code_interpreter"}]
            }
      ]
    }
    if file_ids:
        params["attachments"]=[
            {
            "file_id": file_ids,
            "tools": [{"type": "code_interpreter"}]
            }
      ]

    client.beta.threads.messages.create(
        **params
    )
    
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )

def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id)

ModuleNotFoundError: No module named 'jsonaqqa'

## Create the content

In [44]:
financial_data_path = os.path.join(project_path, 'data', 'NotRealCorp_financial_data.json') 
financial_data = pd.read_json(financial_data_path)
financial_data.head(5)

,Year,Quarter,Distribution channel,Revenue ($M),Costs ($M),Customer count,Time
0,2021,Q1,Online Sales,1.50,1.301953,150,2021 Q1
1,2021,Q1,Direct Sales,1.50,1.380809,151,2021 Q1
2,2021,Q1,Retail Partners,1.50,1.348246,152,2021 Q1
3,2021,Q2,Online Sales,1.52,1.308608,152,2021 Q2
4,2021,Q2,Direct Sales,1.52,1.413305,153,2021 Q2


In [45]:
# upload the file so the Assistant can use it 
file = client.files.create(
  file=open(financial_data_path,"rb"),
  purpose='assistants',
)

In [46]:
# create a assistant 
assistant = client.beta.assistants.create(
  instructions="You are a data scientist assistant. When given data and a query, write the proper code and create the proper visualization",
  model="gpt-4-1106-preview",
  tools=[{"type": "code_interpreter"}],
  tool_resources={
    "code_interpreter": {
    "file_ids": [file.id]
    }
  }
)

In [47]:
# create a thread
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Calculate profit (revenue minus cost) by quarter and year, and visualize as a line plot across the distribution channels, where the colors of the lines are green, light red, and light blue",
      "attachments": [
        {
          "file_id": file.id,
          "tools": [{"type": "code_interpreter"}]
        }
      ]
    }
  ]
)

In [48]:
# run the thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

In [49]:
# Get the response fomr the assiatant
messages = client.beta.threads.messages.list(thread_id=thread.id)

In [50]:
import time

while True:
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    try:
        #See if image has been created
        messages.data[0].content[0].image_file
        #Sleep to make sure run has completed
        time.sleep(5)
        print('Plot created!')
        break
    except:
        time.sleep(10)
        print('Assistant still working...')

Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Plot created!


In [51]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
[message.content[0] for message in messages.data]

[ImageFileContentBlock(image_file=ImageFile(file_id='file-ZNGwUBhnRCT4yNiTSXfpEgJB', detail=None), type='image_file'),
 TextContentBlock(text=Text(annotations=[], value="We have successfully calculated the profit and pivoted the DataFrame to show profits for each distribution channel by year and quarter. Now we can move on to visualizing this data as a line plot. We will use green for 'Online Sales', light red for 'Direct Sales', and light blue for 'Retail Partners'. Let's create the line plot."), type='text'),
 TextContentBlock(text=Text(annotations=[], value="The DataFrame is structured correctly with the necessary columns for our analysis. Next, we will perform the following steps:\n\n1. Calculate the profit for each row by subtracting 'Costs ($M)' from 'Revenue ($M)'.\n2. Aggregate the profit by 'Year', 'Quarter', and 'Distribution channel'.\n3. Plot the result as a line plot with the specified colors for each distribution channel.\n\nLet's start by calculating the profit."), type=

In [52]:
import os

# Get the current working directory
current_path = os.getcwd()

print(f"My current working directory is: {current_path}")

My current working directory is: /Users/tianyuliu/Code/llm/llm_project/src/openai/Applications/Assistant_Examples


In [53]:
# Quick helper function to convert our output file to a PNG and save it in the images folder
def convert_file_to_png(file_id, write_path):
    # Ensure the "images" directory exists
    # os.makedirs(os.path.dirname(write_path), exist_ok=True)
    
    # Fetch the data from the file (generated by LLM) and save it as a PNG
    data = client.files.content(file_id)
    data_bytes = data.read()
    with open(write_path, "wb") as file:
        file.write(data_bytes)

# Create the path to the "images" folder and the PNG file inside it
image_path = os.path.join(project_path, 'images', 'NotRealCorp_chart.png')
# Convert the file and save it to the "images" folder
plot_file_id = messages.data[0].content[0].image_file.file_id
convert_file_to_png(plot_file_id, image_path)

# Upload the saved PNG file
plot_file = client.files.create(
  file=open(image_path, "rb"),
  purpose='assistants'
)

## Generating insights

In [54]:
submit_message(assistant.id,thread,"Give me two medium length sentences (~20-30 words per sentence) of the \
      most important insights from the plot you just created.\
             These will be used for a slide deck, and they should be about the\
                     'so what' behind the data."
)

Run(id='run_Mk2AmIZ4OE7k4HYUqxYZPVPM', assistant_id='asst_lZc0n4bHFioqNjugRuhvSRYG', cancelled_at=None, completed_at=None, created_at=1727078679, expires_at=1727079279, failed_at=None, incomplete_details=None, instructions='You are a data scientist assistant. When given data and a query, write the proper code and create the proper visualization', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_AkDuw8x1kAGwsHesSikB5NTu', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})

In [55]:
# Hard coded wait for a response, as the assistant may iterate on the bullets.
time.sleep(10)
response = get_response(thread)
bullet_points = response.data[0].content[0].text.value
print(bullet_points)

The line plot reveals that 'Online Sales' consistently outperforms the other channels in profit, exhibiting substantial growth year over year, suggestive of a successful and expanding online market presence. 'Direct Sales', while showing a gradual increase, trails behind the other channels, indicating a need to reassess strategies or strengthen customer engagement in this area.


In [56]:
submit_message(assistant.id,thread,"Given the plot and bullet points you created,\
 come up with a very brief title for a slide. It should reflect just the main insights you came up with."
)

Run(id='run_q0m7JAISggVcnLybwCUK5OFi', assistant_id='asst_lZc0n4bHFioqNjugRuhvSRYG', cancelled_at=None, completed_at=None, created_at=1727078693, expires_at=1727079293, failed_at=None, incomplete_details=None, instructions='You are a data scientist assistant. When given data and a query, write the proper code and create the proper visualization', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_AkDuw8x1kAGwsHesSikB5NTu', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})

In [57]:
#Wait as assistant may take a few steps
time.sleep(10)
response = get_response(thread)
title = response.data[0].content[0].text.value
print(title)

"Rising Dominance of Online Sales and Call to Action for Direct Sales"


## 3. DALL E-3 title image

In [58]:
company_summary = "NotReal Corp is a prominent hardware company that manufactures and sells processors, graphics cards and other essential computer hardware."


response = client.images.generate(
  model='dall-e-3',
  prompt=f"given this company summary {company_summary}, create an inspirational \
    photo showing the growth and path forward. This will be used at a quarterly\
       financial planning meeting",
       size="1024x1024",
       quality="hd",
       n=1
)
image_url = response.data[0].url


now we can add this image to our thread. First, we can save the image locally, then upload it to the assistants API using the File upload endpoint

In [59]:

dalle_img_path = os.path.join(project_path, 'images', 'dalle_image.png')
# dalle_img_path = '../images/dalle_image.png'
img = requests.get(image_url)

#Save locally
with open(dalle_img_path,'wb') as file:
  file.write(img.content)

#Upload
dalle_file = client.files.create(
  file=open(dalle_img_path, "rb"),
  purpose='assistants'
)

## 4. Creating the slides

In [60]:
title_template = """
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.dml.color import RGBColor

# Create a new presentation object
prs = Presentation()

# Add a blank slide layout
blank_slide_layout = prs.slide_layouts[6]
slide = prs.slides.add_slide(blank_slide_layout)

# Set the background color of the slide to black
background = slide.background
fill = background.fill
fill.solid()
fill.fore_color.rgb = RGBColor(0, 0, 0)

# Add image to the left side of the slide with a margin at the top and bottom
left = Inches(0)
top = Inches(0)
height = prs.slide_height
width = prs.slide_width * 3/5
pic = slide.shapes.add_picture(image_path, left, top, width=width, height=height)

# Add title text box positioned higher
left = prs.slide_width * 3/5
top = Inches(2)
width = prs.slide_width * 2/5
height = Inches(1)
title_box = slide.shapes.add_textbox(left, top, width, height)
title_frame = title_box.text_frame
title_p = title_frame.add_paragraph()
title_p.text = title_text
title_p.font.bold = True
title_p.font.size = Pt(38)
title_p.font.color.rgb = RGBColor(255, 255, 255)
title_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER

# Add subtitle text box
left = prs.slide_width * 3/5
top = Inches(3)
width = prs.slide_width * 2/5
height = Inches(1)
subtitle_box = slide.shapes.add_textbox(left, top, width, height)
subtitle_frame = subtitle_box.text_frame
subtitle_p = subtitle_frame.add_paragraph()
subtitle_p.text = subtitle_text
subtitle_p.font.size = Pt(22)
subtitle_p.font.color.rgb = RGBColor(255, 255, 255)
subtitle_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER
"""

data_vis_template = """
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.dml.color import RGBColor

# Create a new presentation object
prs = Presentation()

# Add a blank slide layout
blank_slide_layout = prs.slide_layouts[6]
slide = prs.slides.add_slide(blank_slide_layout)

# Set the background color of the slide to black
background = slide.background
fill = background.fill
fill.solid()
fill.fore_color.rgb = RGBColor(0, 0, 0)

# Define placeholders
image_path = data_vis_img
title_text = "Maximizing Profits: The Dominance of Online Sales & Direct Sales Optimization"
bullet_points = "• Online Sales consistently lead in profitability across quarters, indicating a strong digital market presence.\n• Direct Sales show fluctuations, suggesting variable performance and the need for targeted improvements in that channel."

# Add image placeholder on the left side of the slide
left = Inches(0.2)
top = Inches(1.8)
height = prs.slide_height - Inches(3)
width = prs.slide_width * 3/5
pic = slide.shapes.add_picture(image_path, left, top, width=width, height=height)

# Add title text spanning the whole width
left = Inches(0)
top = Inches(0)
width = prs.slide_width
height = Inches(1)
title_box = slide.shapes.add_textbox(left, top, width, height)
title_frame = title_box.text_frame
title_frame.margin_top = Inches(0.1)
title_p = title_frame.add_paragraph()
title_p.text = title_text
title_p.font.bold = True
title_p.font.size = Pt(28)
title_p.font.color.rgb = RGBColor(255, 255, 255)
title_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER

# Add hardcoded "Key Insights" text and bullet points
left = prs.slide_width * 2/3
top = Inches(1.5)
width = prs.slide_width * 1/3
height = Inches(4.5)
insights_box = slide.shapes.add_textbox(left, top, width, height)
insights_frame = insights_box.text_frame
insights_p = insights_frame.add_paragraph()
insights_p.text = "Key Insights:"
insights_p.font.bold = True
insights_p.font.size = Pt(24)
insights_p.font.color.rgb = RGBColor(0, 128, 100)
insights_p.alignment = PP_PARAGRAPH_ALIGNMENT.LEFT
insights_frame.add_paragraph()


bullet_p = insights_frame.add_paragraph()
bullet_p.text = bullet_points
bullet_p.font.size = Pt(12)
bullet_p.font.color.rgb = RGBColor(255, 255, 255)
bullet_p.line_spacing = 1.5
"""

In [61]:
title_text = "NotRealCorp"
subtitle_text = "Quarterly financial planning meeting, Q3 2023"

In [62]:
submit_message(assistant.id,thread,f"Use the included code template to create a PPTX slide that follows the template format, but uses the image, company name/title, and document name/subtitle included:\
{title_template}. IMPORTANT: Use the image file included in this message as the image_path image in this first slide, and use the Company Name {title_text} as the title_text variable, and \
  use the subtitle_text {subtitle_text} a the subtitle_text variable. \
    NEST, create a SECOND slide using the following code template: {data_vis_template} to create a PPTX slide that follows the template format, but uses the company name/title, and document name/subtitle included:\
{data_vis_template}. IMPORTANT: Use the line plot image, that is the second attached image in this message, that you created earlier in the thread as the data_vis_img image, and use the data visualization title that you created earlier for the variable title_text, and\
  the bullet points of insights you created earlier for the bullet_points variable. Output these TWO SLIDES as a .pptx file. Make sure the output is two slides, with each slide matching the respective template given in this message.",
              file_ids=[dalle_file.id, plot_file.id]
)

TypeError: Messages.create() got an unexpected keyword argument 'file_ids'